In [ ]:
from src.data.data_loader import get_train_dataset
from torchvision import datasets, transforms
import torch.nn as nn
import pprint
import torch
import torch.optim as optim
import torch.nn.functional as F
from src.models.model import ShapeClassifier

from src.configs.model_config import ModelConfig
from src.data.data_loader import train_loader, num_classes
from src.utils.logs import writer
from src.utils.train import train
from src.utils.test import test
import wandb
import json

In [ ]:
wandb.login()

In [ ]:
sweep_config = {
    'method': 'bayes',
    "metric": {
        "name": 'accuracy',
        "goal": 'maximize'
    },
    "parameters": {
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'uniform',
            'min': 0,
            'max': 0.001
        },
        'batch_size': {
            # integers between 32 and 256
            # with evenly-distributed logarithms
            'distribution': 'q_log_uniform_values',
            'q': 8,
            'min': 32,
            'max': 256,
        },
        'epochs': {
            'values': [15, 20, 30]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'fc_layer_size': {
            'values': [128, 256, 512]
        },
    }
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
def build_dataset(batch_size):

    return get_train_dataset(batch_size)


def build_network(fc_layer_size):
    network = ShapeClassifier(num_classes=3,hidden_size=fc_layer_size)

    return network.to(device)


def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.cross_entropy(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

def validate(network, loader):
    network.eval()
    correct = 0
    with torch.no_grad():
        for _, (data, target) in enumerate(loader):
            data, target = data.to(device), target.to(device)
            output = network(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    return correct / len(loader.dataset)
    
def train(config=None):
    # Initialize a new wandb run  

    with wandb.init():
        config = wandb.config
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        print("config" , config)
   
        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size)
        optimizer = build_optimizer(
            network, config.optimizer, config.learning_rate)


        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            acc = validate(network, loader)
            print(f"Epoch {epoch} avg loss: {avg_loss} accuracy: {acc}")
            wandb.log({"loss": avg_loss, "epoch": epoch, "accuracy": acc})

In [14]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: 04b419v1
Sweep URL: http://localhost:8080/nguyen/pytorch-sweeps-demo/sweeps/04b419v1


In [16]:
wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: dmjiyx7j with config:
wandb: 	batch_size: 48
wandb: 	epochs: 20
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 9.930082550267504e-06
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


config {'batch_size': 48, 'epochs': 20, 'fc_layer_size': 256, 'learning_rate': 9.930082550267504e-06, 'optimizer': 'adam'}
Epoch 0 avg loss: 1.1227594017982483 accuracy: 0.51
Epoch 1 avg loss: 1.0263650843075343 accuracy: 0.5933333333333334
Epoch 2 avg loss: 0.9231991171836853 accuracy: 0.49
Epoch 3 avg loss: 0.8516879166875567 accuracy: 0.62
Epoch 4 avg loss: 0.8165683661188398 accuracy: 0.81
Epoch 5 avg loss: 0.7675495403153556 accuracy: 0.6733333333333333
Epoch 6 avg loss: 0.7831985950469971 accuracy: 0.7166666666666667
Epoch 7 avg loss: 0.6932210155895778 accuracy: 0.8733333333333333
Epoch 8 avg loss: 0.6585122261728559 accuracy: 0.7333333333333333
Epoch 9 avg loss: 0.6423001033919198 accuracy: 0.8766666666666667
Epoch 10 avg loss: 0.6038253137043544 accuracy: 0.7933333333333333
Epoch 11 avg loss: 0.5986335618155343 accuracy: 0.78
Epoch 12 avg loss: 0.5852176036153521 accuracy: 0.7766666666666666
Epoch 13 avg loss: 0.5776768922805786 accuracy: 0.7733333333333333
Epoch 14 avg loss: 

accuracy,▁▃▁▃▆▄▅▇▅▇▆▆▆▆▆█████
batch loss,█▇▆▅▄▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▃▂▃▂▄▂▃▃▂▃▂▂▂▂▁▁▂▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▆▅▅▄▅▄▃▃▃▃▃▃▂▂▂▁▁▁
accuracy,0.92333
batch loss,0.38374
epoch,19
loss,0.42571


wandb: Agent Starting Run: c61bls20 with config:
wandb: 	batch_size: 120
wandb: 	epochs: 30
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.00040770515026138955
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


config {'batch_size': 120, 'epochs': 30, 'fc_layer_size': 512, 'learning_rate': 0.00040770515026138955, 'optimizer': 'adam'}
Epoch 0 avg loss: 12.669246673583984 accuracy: 0.3333333333333333
Epoch 1 avg loss: 9.699127753575643 accuracy: 0.3933333333333333
Epoch 2 avg loss: 5.997192939122518 accuracy: 0.5666666666666667
Epoch 3 avg loss: 5.1969099044799805 accuracy: 0.5933333333333334
Epoch 4 avg loss: 1.2817224860191345 accuracy: 0.5533333333333333
Epoch 5 avg loss: 2.5963169733683267 accuracy: 0.57
Epoch 6 avg loss: 0.8057341774304708 accuracy: 0.69
Epoch 7 avg loss: 1.1785928010940552 accuracy: 0.6966666666666667
Epoch 8 avg loss: 0.7000896036624908 accuracy: 0.9033333333333333
Epoch 9 avg loss: 0.3257632553577423 accuracy: 0.8033333333333333
Epoch 10 avg loss: 0.42895398537317914 accuracy: 0.9366666666666666
Epoch 11 avg loss: 0.19974619646867117 accuracy: 0.9566666666666667
Epoch 12 avg loss: 0.18941768010457358 accuracy: 0.9533333333333334
Epoch 13 avg loss: 0.13353885461886725 ac

...

config {'batch_size': 104, 'epochs': 30, 'fc_layer_size': 512, 'learning_rate': 0.0003980820663366078, 'optimizer': 'adam'}
Epoch 0 avg loss: 5.062711517016093 accuracy: 0.3433333333333333
Epoch 1 avg loss: 3.3943236668904624 accuracy: 0.7133333333333334
Epoch 2 avg loss: 0.8393203417460123 accuracy: 0.5833333333333334
Epoch 3 avg loss: 1.0142684777577717 accuracy: 0.87
Epoch 4 avg loss: 0.3504032492637634 accuracy: 0.8433333333333334
Epoch 5 avg loss: 0.35366151730219525 accuracy: 0.8866666666666667
Epoch 6 avg loss: 0.25075238943099976 accuracy: 0.9266666666666666
Epoch 7 avg loss: 0.16470391551653543 accuracy: 0.96
Epoch 8 avg loss: 0.11225331077973048 accuracy: 0.9666666666666667
Epoch 9 avg loss: 0.0980972337226073 accuracy: 0.97
Epoch 10 avg loss: 0.08187093834082286 accuracy: 0.99
Epoch 11 avg loss: 0.05791024987896284 accuracy: 0.9966666666666667
Epoch 12 avg loss: 0.0403053325911363 accuracy: 1.0
Epoch 13 avg loss: 0.032867188875873886 accuracy: 1.0
Epoch 14 avg loss: 0.026567

accuracy,▁▅▄▇▆▇▇███████████████████████
batch loss,▂█▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▆▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
batch loss,0.00056
epoch,29
loss,0.00065


wandb: Agent Starting Run: f90ox7w8 with config:
wandb: 	batch_size: 48
wandb: 	epochs: 20
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.0009607460347692044
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


config {'batch_size': 48, 'epochs': 20, 'fc_layer_size': 512, 'learning_rate': 0.0009607460347692044, 'optimizer': 'adam'}
Epoch 0 avg loss: 25.255474090576172 accuracy: 0.5933333333333334
Epoch 1 avg loss: 5.102003284863064 accuracy: 0.5866666666666667
Epoch 2 avg loss: 0.8636805542877742 accuracy: 0.7366666666666667
Epoch 3 avg loss: 0.6188298123223441 accuracy: 0.8166666666666667
Epoch 4 avg loss: 0.27566932354654583 accuracy: 0.9366666666666666
Epoch 5 avg loss: 0.12154114033494677 accuracy: 0.9666666666666667
Epoch 6 avg loss: 0.05851750440030758 accuracy: 0.9966666666666667
Epoch 7 avg loss: 0.030152809540075914 accuracy: 1.0
Epoch 8 avg loss: 0.01572604657017759 accuracy: 1.0
Epoch 9 avg loss: 0.008001219414706742 accuracy: 1.0
Epoch 10 avg loss: 0.0068940040988049334 accuracy: 1.0
Epoch 11 avg loss: 0.0046819809358567 accuracy: 1.0
Epoch 12 avg loss: 0.003610321969193007 accuracy: 1.0
Epoch 13 avg loss: 0.0037284358737191985 accuracy: 1.0
Epoch 14 avg loss: 0.002716043115859585

accuracy,▁▁▄▅▇▇██████████████
batch loss,█▇▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
batch loss,0.00085
epoch,19
loss,0.00169
